In [1]:
# Import library numpy, pandas dan scikit-learn
import numpy as np
import pandas as pd
from sklearn import tree

In [2]:
# Membaca Dataset dari File ke Pandas dataFrame
Dataset = pd.read_csv('dataset/fitness.csv')

In [3]:
print(Dataset.head())

                epoch (ms)  Accelerometer_x  Accelerometer_y  Accelerometer_z  \
0  2019-01-11 15:08:05.200         0.013500         0.977000        -0.071000   
1  2019-01-11 15:08:05.400        -0.001500         0.970500        -0.079500   
2  2019-01-11 15:08:05.600         0.001333         0.971667        -0.064333   
3  2019-01-11 15:08:05.800        -0.024000         0.957000        -0.073500   
4  2019-01-11 15:08:06.000        -0.028000         0.957667        -0.115000   

   Gyroscope_x  Gyroscope_y  Gyroscope_z Participants  Label Category  Set  
0      -1.8904       2.4392       0.9388            B  bench    heavy   30  
1      -1.6826      -0.8904       2.1708            B  bench    heavy   30  
2       2.5608      -0.2560      -1.4146            B  bench    heavy   30  
3       8.0610      -4.5244      -2.0730            B  bench    heavy   30  
4       2.4390      -1.5486      -3.6098            B  bench    heavy   30  


In [4]:
# Menghapus kolom "epoch (ms)"
Dataset = Dataset.drop(labels="epoch (ms)", axis=1)

In [5]:
print(Dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9009 entries, 0 to 9008
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Accelerometer_x  9009 non-null   float64
 1   Accelerometer_y  9009 non-null   float64
 2   Accelerometer_z  9009 non-null   float64
 3   Gyroscope_x      9009 non-null   float64
 4   Gyroscope_y      9009 non-null   float64
 5   Gyroscope_z      9009 non-null   float64
 6   Participants     9009 non-null   object 
 7   Label            9009 non-null   object 
 8   Category         9009 non-null   object 
 9   Set              9009 non-null   int64  
dtypes: float64(6), int64(1), object(3)
memory usage: 704.0+ KB
None


In [6]:
# Mengubah kelas (kolom "Lung Canser") dari String ke Unique-Integer
Dataset["Category"] = pd.factorize(Dataset.Category)[0]

In [7]:
# Mengubah dataFrame ke array Numpy
Dataset = Dataset.to_numpy()

In [8]:
# #Membagi Dataset => 80 baris data untuk training dan 20 baris data untuk testing
# dataTraining = np.concatenate((Dataset[0:40, :], Dataset[50:90, :]), 
#                               axis=0)
# dataTesting = np.concatenate((Dataset[40:50, :], Dataset[90:100, :]), 
#                              axis=0)


In [9]:
from sklearn.model_selection import train_test_split
# Membagi dataset menjadi 80% untuk pelatihan dan 20% untuk pengujian
dataTraining, dataTesting = train_test_split(Dataset, test_size=0.2, random_state=0)

# Menampilkan bentuk dataTraining dan dataTesting
print("Bentuk dataTraining:", dataTraining.shape)  
print("Bentuk dataTesting:", dataTesting.shape)   

Bentuk dataTraining: (7207, 10)
Bentuk dataTesting: (1802, 10)


In [10]:
# Memecah Dataset ke Input dan Label
inputTraining = dataTraining[:, 0:9]  # Mengambil semua kolom input (kolom 1 sampai 9)
inputTesting = dataTesting[:, 0:9]
labelTraining = dataTraining[:, 8]  # Kolom ke-9 sebagai label
labelTesting = dataTesting[:, 8]     # Kolom ke-9 sebagai label

In [11]:
print(inputTraining)

[[-0.2263333333333333 1.074 -0.0303333333333333 ... 'A' 'ohp' 1]
 [0.031 -1.092 -0.073 ... 'E' 'dead' 1]
 [0.9743333333333334 -0.2733333333333334 -0.1286666666666666 ... 'A'
  'rest' 2]
 ...
 [0.097 -1.1435 -0.0345 ... 'A' 'dead' 1]
 [0.1566666666666666 -1.1506666666666667 0.157 ... 'C' 'row' 0]
 [-0.352 0.8290000000000001 -0.2066666666666666 ... 'A' 'ohp' 0]]


In [12]:
print(labelTraining)

[1 1 2 ... 1 0 0]


In [13]:
# Membulatkan labelTraining
labelTraining = labelTraining.astype(int)

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [6,7])], remainder='passthrough')
# Transformasi inputTraining dan inputTesting
inputTraining = np.array(ct.fit_transform(inputTraining))
inputTesting = np.array(ct.transform(inputTesting))  # Hanya transformasi, tidak fit ulang

# Menampilkan hasil
print("Input Training:\n", inputTraining[:5])  # Tampilkan 5 baris pertama
print("Input Testing:\n", inputTesting[:5]) 

Input Training:
 [[1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 -0.2263333333333333 1.074
  -0.0303333333333333 49.317 23.2806 -53.829600000000006 1]
 [0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.031 -1.092 -0.073 9.0856
  -8.2806 0.6462 1]
 [1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.9743333333333334
  -0.2733333333333334 -0.1286666666666666 4.6222 18.7562 -1.2196 2]
 [0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 -0.259 0.872
  0.4106666666666667 -8.1098 -4.8902 -1.7806000000000002 1]
 [0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 -0.037 -1.030333333333333
  -0.0533333333333333 -2.7684 -0.5854 2.244 1]]
Input Testing:
 [[0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.075 -1.4720000000000002
  -0.107 40.0366 -38.0852 -22.988 0]
 [0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 -0.422 0.9506666666666668
  -0.385 2.634 8.4878 8.4758 1]
 [1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0856666666666666
  -0.8543333333333334 0.4796666666666667 59.7682 -60.0486 -29.0366 3]
 [0.0 0.0 1.0 0.0 0.0 

In [15]:
# Mendefinisikan Decision Tree Classifier
model = tree.DecisionTreeClassifier()

In [16]:
# Mentraining Model
model = model.fit(inputTraining, labelTraining)

In [17]:
# Memprediksi Input Data Testing
hasilPrediksi = model.predict(inputTesting)
print("Label Sebenarnya : ", labelTesting)
print("Hasil Prediksi : ", hasilPrediksi)

Label Sebenarnya :  [0 1 3 ... 0 0 1]
Hasil Prediksi :  [0 1 3 ... 0 0 1]


In [18]:
# Menghitung Akurasi
prediksiBenar = (hasilPrediksi == labelTesting).sum()
prediksiSalah = (hasilPrediksi != labelTesting).sum()
print("Prediksi Benar :", prediksiBenar, "data")
print("Prediksi Salah :", prediksiSalah, "data")
print("Akurasi :", prediksiBenar/(prediksiBenar+prediksiSalah) * 100, "%")

Prediksi Benar : 1802 data
Prediksi Salah : 0 data
Akurasi : 100.0 %
